In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_65462/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 04:03:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/18 04:03:57 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [21]:
import pandas as pd

# Sample data for Keywords table
keywords_data = {
    'topic_id': [1, 1, 3, 2],
    'word': ['handball', 'football', 'WAR', 'Vaccine']
}

# Sample data for Posts table
posts_data = {
    'post_id': [1, 2, 3, 4],
    'content': [
        'We call it soccer. They call it football. Hahaha.',
        'Americans prefer basketball while Europeans love handball and football.',
        'Stop the war and play handball.',
        'Warning: I planted some flowers this morning and then got vaccinated.'
    ]
}

# Create DataFrame for Keywords
keywords_df = pd.DataFrame(keywords_data)

# Create DataFrame for Posts
posts_df = pd.DataFrame(posts_data)

# Display the DataFrames
print("Keywords table:")
print(keywords_df)
print("\nPosts table:")
print(posts_df)



df_person = spark.createDataFrame(keywords_df)
df_person.createOrReplaceTempView("Keywords")

df_person = spark.createDataFrame(posts_df)
df_person.createOrReplaceTempView("Posts")



Keywords table:
   topic_id      word
0         1  handball
1         1  football
2         3       WAR
3         2   Vaccine

Posts table:
   post_id                                            content
0        1  We call it soccer. They call it football. Hahaha.
1        2  Americans prefer basketball while Europeans lo...
2        3                    Stop the war and play handball.
3        4  Warning: I planted some flowers this morning a...


In [22]:
spark.sql("select *, instr(content, 'football') from Posts").show(truncate = False)

+-------+-----------------------------------------------------------------------+------------------------+
|post_id|content                                                                |instr(content, football)|
+-------+-----------------------------------------------------------------------+------------------------+
|1      |We call it soccer. They call it football. Hahaha.                      |33                      |
|2      |Americans prefer basketball while Europeans love handball and football.|63                      |
|3      |Stop the war and play handball.                                        |0                       |
|4      |Warning: I planted some flowers this morning and then got vaccinated.  |0                       |
+-------+-----------------------------------------------------------------------+------------------------+



In [31]:
query = """
    select
        p.post_id,
        concat_ws(',', collect_list(k.topic_id))
    from Posts p left join Keywords k on lower(p.content) like concat('% ', lower(k.word), ' %')
    group by p.post_id
"""

In [32]:
spark.sql(query).show()

+-------+------------------------------------+
|post_id|concat_ws(,, collect_list(topic_id))|
+-------+------------------------------------+
|      3|                                   3|
|      4|                                    |
|      1|                                    |
|      2|                                   1|
+-------+------------------------------------+

